In [ ]:
!pip install openai
import openai
openai.api_key = 'OpenAIAPI金鑰'
GPT_MODLE = 'gpt-3.5-turbo-0125'

!pip install gradio
import gradio as gr

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
with open("finetune.jsonl", "rb") as training_fd:
  training_response = openai.files.create(
    file = training_fd,
    purpose = "fine-tune"
  )

training_file_id = training_response.id

print("Training file ID:", training_file_id)

In [ ]:
response = openai.fine_tuning.jobs.create(
    training_file = training_file_id,
    model = GPT_MODLE,
    suffix = "my_model",
)

job_id = response.id

print("Job ID:", response.id)
print("Status:", response.status)

In [ ]:
response = openai.fine_tuning.jobs.retrieve(job_id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

In [ ]:
response = openai.fine_tuning.jobs.list_events(job_id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

In [ ]:
response = openai.fine_tuning.jobs.retrieve(job_id)
fine_tuned_model_id = response.fine_tuned_model

if fine_tuned_model_id is None:
    raise RuntimeError("未找到微調的模型 ID。 你的工作可能還沒完成。")

print("Fine-tuned model ID:", fine_tuned_model_id)

In [ ]:
def chat(query):
  response = openai.chat.completions.create(
    model = fine_tuned_model_id,
    messages = [
        {"role":"user","content":query}
    ]
  )
  return response.choices[0].message.content

In [ ]:
gr.Interface(
    fn = chat,
    inputs = gr.Textbox(label="輸入您的問題："),
    outputs = gr.Textbox(label="AI 的回答："),
    title = "AI 聊天室",
    allow_flagging = "never"
).queue().launch()